In [1]:
import os
import numpy as np
import pandas as pd
!pip install music21
from music21 import converter, instrument, note, chord, midi, stream
from keras.models import Model, Sequential
from keras.layers import *
from keras.models import load_model
import keras.backend as K
import keras.callbacks
from keras.utils import to_categorical
from keras.preprocessing.sequence import TimeseriesGenerator

You should consider upgrading via the 'python -m pip install --upgrade pip' command.
Using TensorFlow backend.


In [3]:
from google.colab import files
files.upload()

Saving Sor_Study_no1.mid to Sor_Study_no1.mid
Saving Sor_Study_no2.mid to Sor_Study_no2.mid
Saving Sor_Study_no3.mid to Sor_Study_no3.mid
Saving Sor_Study_no4.mid to Sor_Study_no4.mid
Saving Sor_Study_no5.mid to Sor_Study_no5.mid
Saving Sor_Study_no6.mid to Sor_Study_no6.mid
Saving Sor_Study_no7.mid to Sor_Study_no7.mid
Saving Sor_Study_no8.mid to Sor_Study_no8.mid
Saving Sor_Study_no9.mid to Sor_Study_no9.mid
Saving Sor_Study_no10.mid to Sor_Study_no10.mid
Saving Sor_Study_no11.mid to Sor_Study_no11.mid
Saving Sor_Study_no12.mid to Sor_Study_no12.mid
Saving Sor_Study_no13.mid to Sor_Study_no13.mid
Saving Sor_Study_no14.mid to Sor_Study_no14.mid
Saving Sor_Study_no15.mid to Sor_Study_no15.mid
Saving Sor_Study_no16.mid to Sor_Study_no16.mid
Saving Sor_Study_no17.mid to Sor_Study_no17.mid
Saving Sor_Study_no18.mid to Sor_Study_no18.mid
Saving Sor_Study_no19.mid to Sor_Study_no19.mid
Saving Sor_Study_no20a.mid to Sor_Study_no20a.mid


{'Sor_Study_no1.mid': b'MThd\x00\x00\x00\x06\x00\x01\x00\x04\x01\xe0MTrk\x00\x00\x00\x19\x00\xffQ\x03\x0bq\xb0\x00\xffX\x04\x03\x02\x18\x08\x00\xffY\x02\x00\x00\x00\xff/\x00MTrk\x00\x00\x06\x1e\x00\xff\x03\x00\x00\xff!\x01\x00\x00\xff \x01\x01\x00\xff \x01\x02\x00\xb0\x07\x7f\x00\xb1\x07\x7f\x00\xb0]\x00\x00\xb1]\x00\x00\xb0[\x7f\x00\xb1[\x7f\x00\xb0\n@\x00\xb1\n@\x00\xc0\x18\x00\xc1\x18\x00\x90@\x7f\x00\x90<\x7f\x00\x900\x7f\x83`\x80@P\x00\x80<P\x00\x800P\x00\x90@\x7f\x00\x90<\x7f\x00\x904\x7f\x83`\x80@P\x00\x80<P\x00\x804P\x00\x90@\x7f\x00\x90<\x7f\x00\x905\x7f\x83`\x80@P\x00\x80<P\x00\x805P\x00\x90>\x7f\x00\x907\x7f\x87@\x80>P\x00\x807P\x83`\x90A\x7f\x00\x90>\x7f\x00\x902\x7f\x83`\x80AP\x00\x80>P\x00\x802P\x00\x90A\x7f\x00\x90>\x7f\x00\x905\x7f\x83`\x80AP\x00\x80>P\x00\x805P\x00\x90A\x7f\x00\x90>\x7f\x00\x907\x7f\x83`\x80AP\x00\x80>P\x00\x807P\x00\x90@\x7f\x00\x909\x7f\x87@\x80@P\x00\x809P\x83`\x90C\x7f\x00\x90@\x7f\x00\x904\x7f\x83`\x80CP\x00\x80@P\x00\x804P\x00\x90@\x7f\x00\x90<\x

In [0]:
!mkdir guitar_sor
!mv *.mid guitar_sor/.

In [42]:
x_notes = []
y_notes = []
track = 0
duration = 1 / 4

directory = "midi/band/"
for song_dir in os.listdir(directory):
    for i, file in enumerate(os.listdir(os.path.join(directory, song_dir))):
        midi_part = converter.parse(os.path.join(directory, song_dir, file))
        # Use only first track
        midi_part = midi_part[track]
        notes_to_parse = None
        # Parse the midi file by the notes/chords it contains
        notes_to_parse = midi_part.flat.notesAndRests
        last_elem = note.Rest()
        for elem in notes_to_parse:
            
            d = elem.duration.quarterLength
            
            if isinstance(elem, note.Note):
                """if elem == last_elem:
                    if "Drum" in file:
                        y_notes.append(["rest"])
                    else:
                        x_notes.append(["rest"]) """
                
                while d >= 0:
                    if "Drum" not in file:
                        x_notes.append([str(elem.pitch)])
                    else:
                        y_notes.append([str(elem.pitch)])
                    d -= duration
                    
            elif isinstance(elem, chord.Chord):
                """if elem == last_elem:
                    if "Drum" in file:
                        y_notes.append(["rest"])
                    else:
                        x_notes.append(["rest"])"""

                while d >= 0:
                    if "Drum" not in file:
                        x_notes.append([str(n.nameWithOctave) for n in elem.pitches])
                    else:
                        y_notes.append([str(n.nameWithOctave) for n in elem.pitches])
                    d -= duration
                    
            elif isinstance(elem, note.Rest):

                while d >= 0:
                    if "Drum" not in file:
                        x_notes.append([elem.name])
                    else:
                        y_notes.append([elem.name])    
                    d -= duration
                    
            last_elem = elem
            
        for i in range(int(1 // duration)):
            if "Drum" not in file:
                x_notes.append(["rest"])
            else:
                y_notes.append(["rest"])   
        print("Song %s Loaded" % file)
                
print("DONE LOADING SONGS")    
# Get all pitch names
x_np_notes = np.array(x_notes)
y_np_notes = np.array(y_notes)
x_np_notes = x_np_notes.flatten()
y_np_notes = y_np_notes.flatten()
x_vocab = []
y_vocab = []
for item in x_np_notes:
    #if len(item) > 1 and item != "rest":
    #    x_vocab.append(",".join([note for note in item]))
    #else:
        x_vocab.append(item)
for item in y_np_notes:
    if len(item) > 1 and item != "rest":
        y_vocab.append(",".join([note for note in item]))
    else:
        y_vocab.append(item[0])
x_vocab = sorted(set(x_vocab))
y_vocab = sorted(set(y_vocab))
# Get all notes names
x_vocab_length = len(x_vocab)
y_vocab_length = len(y_vocab)



print(x_vocab_length, x_vocab, len(x_notes), x_notes)
print(y_vocab_length, y_vocab, len(y_notes), y_notes)

Song 066 6-8 Ballad Verse.mid Loaded
Song Drums - 066 6-8 Verse 2 Stick F1.mid Loaded
Song 066 6-8 Ballad Chorus.mid Loaded
Song Drums - 066 6-8 Chorus 1 Ride F5.mid Loaded
Song 080 12-8 Slow Low Down.mid Loaded
Song Drums - 080 12-8 Slow Hat F1.mid Loaded
Song 090 Funk Rock A ALT.mid Loaded
Song Drums - 090 S09 Verse F1.mid Loaded
Song 090 Funk Rock A.mid Loaded
Song Drums - 090 S09 Verse F1.mid Loaded
Song 090 Funk Rock B ALT.mid Loaded
Song Drums - 090 S09 Verse F1.mid Loaded
Song 090 Funk Rock B.mid Loaded
Song Drums - 090 S09 Verse F1.mid Loaded
Song 115 Jazz 2-Beat ALT.mid Loaded
Song Drums - 113 Basic Swing 01a.mid Loaded
Song 115 Jazz 2-Beat.mid Loaded
Song Drums - 113 Basic Swing 01a.mid Loaded
Song 115 Jazz Walk ALT.mid Loaded
Song Drums - 113 Basic Swing 01a.mid Loaded
Song 115 Jazz Walk.mid Loaded
Song Drums - 113 Basic Swing 01a.mid Loaded
Song 140 Half-Time A1.mid Loaded
Song Drums - 140 HT 01 QTR OH.mid Loaded
Song 140 Half-Time A2.mid Loaded
Song Drums - 140 HT 01 QTR O

In [43]:
# turn notes to integers:
x_int_notes = []
y_int_notes = []
for elem in x_notes:
    x_int_notes.append(x_vocab.index(",".join(elem)))
for elem in y_notes:
    y_int_notes.append(y_vocab.index(",".join(elem)))
# onehot encode
x_cat_notes = to_categorical(x_int_notes)
y_cat_notes = to_categorical(y_int_notes)
print(x_cat_notes, y_cat_notes)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]] [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [45]:
seq_len = 128
batch_size = 32
split = int(0.8 * len(x_cat_notes)) // batch_size

train_data_gen = TimeseriesGenerator(x_cat_notes[split:], y_cat_notes[split:],
                               length=seq_len, sampling_rate=1,
                               batch_size=batch_size)
test_data_gen = TimeseriesGenerator(x_cat_notes[:split], y_cat_notes[:split],
                               length=seq_len, sampling_rate=1,
                               batch_size=batch_size)

ValueError: Data and targets have to be of same length. Data length is 2975 while target length is 3713

In [8]:
x, y = train_data_gen[0]
x.shape, y.shape

((32, 128, 387), (32, 387))

In [15]:
K.clear_session()

in_seq = Input(shape=(seq_len, len(vocab)))

x = Bidirectional(LSTM(32))(in_seq)
x = Dropout(0.3)(x)
out = Dense(len(vocab), activation='softmax')(x)

model = Model(inputs=[in_seq], outputs=[out])
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 387)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               1318912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 387)               198531    
Total params: 1,517,443
Trainable params: 1,517,443
Non-trainable params: 0
_________________________________________________________________


In [0]:
#save best model if model improved
model_name = "Sor.h5"
best_checkpoint = keras.callbacks.ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [0]:
h = model.fit_generator(
    train_data_gen,
    steps_per_epoch = len(cat_notes[:split]),
    validation_data = test_data_gen, 
    epochs = 100,
    callbacks=[best_checkpoint])

Epoch 1/100
565/565 [==============================] - 233s 412ms/step - loss: 4.1241 - acc: 0.1786 - val_loss: 2.1202 - val_acc: 0.4783

Epoch 00001: val_loss improved from inf to 2.12022, saving model to Sor.h5
Epoch 2/100
565/565 [==============================] - 227s 402ms/step - loss: 3.3837 - acc: 0.3143 - val_loss: 1.3179 - val_acc: 0.7826

Epoch 00002: val_loss improved from 2.12022 to 1.31790, saving model to Sor.h5
Epoch 3/100
565/565 [==============================] - 227s 402ms/step - loss: 2.9640 - acc: 0.3956 - val_loss: 0.7725 - val_acc: 0.8947

Epoch 00003: val_loss improved from 1.31790 to 0.77253, saving model to Sor.h5
Epoch 4/100
565/565 [==============================] - 227s 401ms/step - loss: 2.5803 - acc: 0.4695 - val_loss: 0.7227 - val_acc: 0.9016

Epoch 00004: val_loss improved from 0.77253 to 0.72266, saving model to Sor.h5
Epoch 5/100
565/565 [==============================] - 227s 401ms/step - loss: 2.3878 - acc: 0.5125 - val_loss: 0.7404 - val_acc: 0.8947

In [0]:
#print History graph
historydf = pd.DataFrame(h.history, index=h.epoch)
historydf.plot(ylim=(0,1))

In [0]:
#model = load_model('Sor.h5')

In [26]:
# Build seed
#idx = -7
#x_test = test_data_gen[idx][0]
x_test = [cat_notes[-seq_len:]]
x_test_stream = stream.Stream()
last_note = ""
for i in range(seq_len):
    str_note = vocab[np.argmax(x_test[0][i])]
    if str_note == last_note:
        continue
    if len(str_note.split(",")) > 1:
        d = 1
        _chord = chord.Chord(str_note.split(","))
        while i+d < len(x_test[0]) and vocab[np.argmax(x_test[0][i+d])] == str_note:
            d += 1
        _chord.duration.quarterLength = d * duration
        x_test_stream.append(_chord)
    else:
        if str_note != "rest":
            d = 1
            while i+d < len(x_test[0]) and vocab[np.argmax(x_test[0][i+d])] == str_note:
                d += 1
            _note = note.Note(str_note)
            _note.duration.quarterLength = d * duration
            x_test_stream.append(_note)
        else:
            d = 1
            while i+d < len(x_test[0]) and vocab[np.argmax(x_test[0][i+d])] == str_note:
                d += 1
            _rest = note.Rest()
            _rest.duration.quarterLength = d * duration
            x_test_stream.append(_rest)
    last_note = str_note
x_test_stream.insert(0, instrument.Guitar())
x_test[0].shape

(128, 387)

In [0]:
# make seq_len predictions from seed
preds = []
x = x_test[0]
for i in range(seq_len // 2):
    pred = model.predict(np.array([x]))
    pred = np.argmax(pred)
    cat_pred = [to_categorical(pred, num_classes=len(vocab))]
    x = x[1:]
    x = list(x) + cat_pred
    preds.append(cat_pred)
print(len(preds))

16


In [0]:
# Build predicted stream
y_test_stream = stream.Stream()
last_note = ""
for i in range(len(preds)):
    str_note = vocab[np.argmax(preds[i])]
    if str_note == last_note:
        continue
    if len(str_note.split(",")) > 1:
        d = 1
        _chord = chord.Chord(str_note.split(","))
        while i+d < len(preds) and vocab[np.argmax(preds[i+d])] == str_note:
            d += 1
        _chord.duration.quarterLength = d * duration
        y_test_stream.append(_chord)
    else:
        if str_note != "rest":
            d = 1
            while i+d < len(preds) and vocab[np.argmax(preds[i+d])] == str_note:
                d += 1
            _note = note.Note(str_note)
            _note.duration.quarterLength = d * duration
            y_test_stream.append(_note)
        else:
            d = 1
            while i+d < len(preds) and vocab[np.argmax(preds[i+d])] == str_note:
                d += 1
            _rest = note.Rest()
            _rest.duration.quarterLength = d * duration
            y_test_stream.append(_rest)
    last_note = str_note
y_test_stream.insert(0, instrument.Guitar())
print(y_test_stream)

<music21.stream.Stream 0x22a340c2b00>


In [27]:
# save streams to midi files
x_test_stream.write("midi", "x_test.mid"), y_test_stream.write("midi", "y_test.mid")

NameError: ignored

In [17]:
# play seed
sp = midi.realtime.StreamPlayer(x_test_stream)
sp.play()

StreamPlayerException: ignored

In [0]:
# play generated music
sp = midi.realtime.StreamPlayer(y_test_stream)
sp.play()

In [0]:
files.download("x_test.mid")